Testing data positive feedback dynamic (20%)


In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')[:100]
all_3374 = np.load('../Data/npy/all_3374D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')[:100]
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')[:100]

In [4]:
#The number of followers for each YouTuber
YouTuber_followers = np.sum(user_following, axis=0)
#print(YouTuber_followers)
over5 = 0
for num in YouTuber_followers:
    if num >= 5:
        over5+=1
print('The num of followers over 5:',over5)

The num of followers over 5: 42


In [5]:
print('user_following shape ',user_following.shape)
print('all_3374 shape ',all_3374.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (100, 88)
all_3374 shape  (88, 3374)
user_category shape  (100, 17)
YouTuber_category shape  (88, 17)


In [6]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (100, 17)


In [7]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)

In [8]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  21


In [9]:
over_10 = 0
over_8 = 0
over_12 = 0
user_idx_over10 = []
avg_over10 = 0
for i in range(len(num_of_follower)):
    num = num_of_follower[i]
    if num >= 10:
        over_10 += 1
        user_idx_over10.append(i)
        avg_over10+=num
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)
print('avg 10:',avg_over10/len(user_idx_over10))

over 8: 68
over 10: 56
over 12: 28
avg 10: 12.517857142857142


# Training data and Testing data

In [10]:
test_amount = 10
yt_test_amount = 20

In [11]:
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(42)
test_idx = random.sample(user_idx,test_amount)
print(test_idx)

[81, 14, 3, 94, 35, 31, 28, 17, 13, 86]


In [12]:
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),int(len(temp_t)/2))
        t_for_test = random.sample(temp_t,math.ceil(len(temp_t)/2))
        #t_for_test = random.sample(temp_t,int(len(temp_t)/2))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [13]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
avg

5.5

In [14]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 100
The length of train_f: 100
The length of test_t: 10
The length of test_f: 10


# Recommendation  Module

In [15]:
#Embedding for feature vector
"""
n: the number of users
m: the number of YouTubers
k: latent dims
l: feature dims
"""
tf.reset_default_graph()
n = len(user_following)
m = 88  
k = 100
l = 3374
embedding_dims = 150

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None,l))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))


image_i = tf.placeholder(tf.float32, shape=(1,l))
image_j = tf.placeholder(tf.float32, shape=(1,l))

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [n, k],
                                      initializer=tf.random_normal_initializer(0,1,seed=3))
    item_latent = tf.get_variable("item_latent", [m, k],
                                      initializer=tf.random_normal_initializer(0,1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [m, k],
                                      initializer=tf.random_normal_initializer(0,1,seed=3))
    W1 = tf.get_variable("W1", [n, k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [n,k,k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [m,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [n,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [n,k,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
    
    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    Embedding = tf.get_variable("embedding", [embedding_dims,l],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Beta = tf.get_variable("beta", [n,embedding_dims],
                             # initializer=tf.contrib.layers.xavier_initializer())
                                     initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i) 
vj = tf.nn.embedding_lookup(item_latent, j)

w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
#wu = Wu
wu = tf.expand_dims(tf.squeeze(tf.nn.embedding_lookup(Wu, user)),0) #(k*k)
wy = tf.expand_dims(tf.squeeze(tf.nn.embedding_lookup(Wy, i)),0) #(k*k)
#wa = Wa
wa = tf.expand_dims(tf.squeeze(tf.nn.embedding_lookup(Wa, user)),0) #(k*k)
#wv = Wv
wv = tf.expand_dims(tf.squeeze(tf.nn.embedding_lookup(Wv, user)),0) #(k,l)

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor
embedding = Embedding

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [22]:
a_list=tf.Variable([])
q = tf.constant(0)
def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])
def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)

    a_list = tf.concat([a_list,(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
            tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
            tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
            tf.matmul(wv, xfi, transpose_b=True)))[0])*r[q]],0)
    q += 1
    return q,  a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])
a_list_soft=tf.nn.softmax(a_list)


aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
q = tf.constant(0)
def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
    q += 1
    return q, aux_np

_,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

"""
for q in range(3): #取q個auxliary item
    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
"""

aux_part = tf.matmul(aux_np, vi, transpose_b=True)
#tf.print('aux attention:',aux_np)
aux_np+=u #user_latent factor + sum (alpha*auxilary)
aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

#矩陣中對應函數各自相乘
# ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

xuij = tf.subtract(xui,xuj)

norm_par = [tf.reduce_sum(tf.multiply(u, u)),tf.reduce_sum(tf.multiply(vi, vi)),tf.reduce_sum(tf.multiply(vj, vj)),
           tf.reduce_sum(tf.multiply(w1, w1)),tf.reduce_sum(tf.multiply(wu, wu)),tf.reduce_sum(tf.multiply(wy, wy)),
           tf.reduce_sum(tf.multiply(wa, wa)),tf.reduce_sum(tf.multiply(wv,wv)),tf.reduce_sum(tf.multiply(beta,beta))]
l2_norm = tf.add_n([
            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.0001 * tf.reduce_sum(tf.multiply(w1, w1)),
            0.0001 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.0001 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.0001 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.0001 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.01 * tf.reduce_sum(tf.multiply(beta,beta)),
            
          ])

loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

ValueError: Shape must be rank 1 but is rank 2 for 'while_3/concat' (op: 'ConcatV2') with input shapes: [?], [1,1], [].

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0=time.time()

#use_true=init_list_of_objects(136)
#use_test=init_list_of_objects(136)

#train_pair_t=[] #positive feedback
#train_pair_f=[] #negative feedback
train_yes_id=[] 
for q in range(5):
    print('Iteraction:',q)
    train_auc=0
    total_loss=0
    xuij_auc=0
    length = 0
    for z in range(n):
        print(z)
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes=[]
        yesr=[]
        
        
        sample=random.sample(train_t[z],len(train_t[z])) #隨機選3個sample true's YouTuber
        train_yes_id.append(sample) #sample全部丟進去
        
        #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
        #change
        r_3=np.zeros(len(sample)) 
        alpha_history = []
        a_list_history = []
        U_history = []
        Y_history = []
        
        #print(len(sample))
        #check if all YouTuber are in train_t or train_f
        #if len(train_t[z])+len(train_f[z]) != 88:
            #print(z,len(train_t[z])+len(train_f[z]))
         
        for b in range(len(sample)):
            yes.append(all_3374[sample[b]])
            yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
            #print('YouTuber_category ', YouTuber_category[sample[k]])
            #print('User_category ',user_category_norm[z])
        #print(yesr)
        
        for b in range(len(yesr)):
            r_3[b]=max(yesr[b])
        #print('r_3:',r_3)
        
        yes=np.array(yes)
        #print('user shape should be ',np.array([z]).shape)
        #print('xf shape should be ',yes.shape)
        #print('r shape should be ',np.array(r_3).shape)
        #print('l_id shape should be ',np.array(sample).shape)
        
        #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
        #取positive 
        train_t_sample = random.sample(train_t[z],len(train_t[z]))
        #print('number of positive feedback', len(train_t_sample))
        
        train_f_sample = random.sample(train_f[z],20)
        for ta in train_t_sample:
            #print(ta,'--> positive feedback')
            
            pos = sample.index(ta)
            #new_sample = np.delete(sample,[pos])
            #new_yes = np.delete(yes,[pos],axis=0)
            #new_r_3 = np.delete(r_3,[pos])
            new_sample = sample
            new_yes = yes
            new_r_3 = r_3
            #print(len(yes),len(new_yes))
            #print(yes)
            #print(new_yes)
            
            
            
            #ta=random.choice(train_t[z]) #ta is true positve photo
            #train_pair_t.append(ta)
            image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
            #print('Image_1 shape ',image_1.shape)
            #train_f_sample = random.sample(train_f[z],20)
            #print('True:',train_t_sample,'Now:',ta)
            #print('False:',train_f_sample)
            for b in train_f_sample:
                #print('likes:',ta,';Not likes:',b)
                #b=random.choice(train_f[z])  #b is no feedback photo
                #train_pair_f.append(b)
                image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                #print('Image_2 shape',image_2.shape)
            
                #use_test[z].append(b)
                _norm_par,_embedding,_a_list,r3,_auc, _loss,_=sess.run([norm_par,embedding,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: new_yes , l_id:new_sample, l_id_len:[len(new_sample)],r:new_r_3,
                                        image_i:image_1,image_j:image_2})
                
                #print(XUIJ)
                #print('loss=',_loss)
                #print('auc=',_auc)
                
                #print('after softmax:',r3)
                #print('before softmax:',_a_list)
                #print('par square:',_norm_par)
                #print('embedding:',_embedding)
                #print('---------------------------------------------------')
                a_list_history.append(_a_list)
                alpha_history.append(r3)
                train_auc+=_auc
                total_loss+=_loss
                length += 1
            #now1+=1
        #final par for this user 
        print('par square:',_norm_par)
        print('before softmax:',_a_list)
        print('after softmax:',r3)
        
        #np.save('../Data/latent_factor/YRM_up10_ALL/Embedding/'+str(q)+'_'+str(z),_embedding)
    
    #print('mine:',xuij_auc/136)    
    #print('a_list_soft:',r3)
    print("total_loss:-----------------", total_loss/length)
    print("train_auc:-------------------", train_auc/length)
    print('---------------------------------------------------------------------------------------------------')
    loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
    print('time:',time.time()-t0,' sec')
print('Total cost ',time.time()-t0,' sec')       

Iteraction: 0
0
par square: [113.02298, 91.30945, 70.66583, 0.76826084, 0.05861775, 0.5825531, 0.05934139, 1.3888856e-06, 0.001756514]
before softmax: [-0.04598401  0.00058401 -0.02582197 -0.00800305 -0.05843479 -0.0060325
  0.05822861]
after softmax: [0.13802679 0.14460643 0.14083792 0.14337    0.13631889 0.1436528
 0.15318716]
1
par square: [97.70038, 82.58603, 89.98308, 0.6831689, 0.0018074594, 0.43634403, 0.0018660632, 5.610282e-07, 0.010830689]
before softmax: [-0.07194728 -0.05345261  0.00521797 -0.03061358 -0.01883766 -0.00019747
 -0.02068912]
after softmax: [0.13656327 0.13911246 0.14751846 0.14232622 0.14401214 0.14672174
 0.14374577]
2
par square: [96.21189, 103.33931, 88.938614, 0.5788333, 2.822987e-06, 0.3311796, 3.0420333e-06, 5.951308e-09, 0.016334884]
before softmax: [ 0.00161293  0.02794115  0.04081074  0.00861615  0.020711    0.0380737
  0.05317106 -0.01752932  0.0226797 ]
after softmax: [0.10886916 0.11177356 0.11322133 0.10963428 0.11096834 0.11291187
 0.11462948 0.1

KeyboardInterrupt: 

In [52]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('time=',loss_acc_list[i][2])

Iteration: 0
loss= [[0.1786452]]
acc= 0.9414904862579281
time= 177.7749536037445
Iteration: 1
loss= [[0.07883559]]
acc= 0.9730443974630021
time= 355.5616991519928
Iteration: 2
loss= [[0.04341662]]
acc= 0.9867864693446089
time= 532.7977256774902
Iteration: 3
loss= [[0.03551585]]
acc= 0.9904862579281184
time= 709.9629302024841
Iteration: 4
loss= [[0.02745813]]
acc= 0.9932875264270613
time= 887.3929221630096


# Get latent factor and Each weight

In [19]:
U, Y, A,Au, Ay, Aa, Av,E,B =sess.run([user_latent, item_latent, aux_item, Wu, Wy, Wa, Wv,embedding,Beta])

In [20]:
print(B)

[[ 5.3598476e-03 -4.5510782e-03  4.5005041e-03 ...  5.0074868e-03
   3.9467365e-03 -4.8622806e-03]
 [-9.1151241e-03  6.2404866e-03 -8.4940586e-03 ... -1.0124578e-02
  -7.2726789e-03  8.7228790e-03]
 [-1.0789812e-02  1.0273932e-02 -1.0842401e-02 ... -1.0192124e-02
  -1.1793047e-02  1.0504904e-02]
 ...
 [ 1.0468982e-05  1.0279150e-05  1.0025165e-05 ...  8.9462610e-06
   1.1183760e-05  8.8296165e-06]
 [ 1.1942506e-05  1.0805446e-05  1.0591112e-05 ...  8.8826464e-06
   9.4343468e-06  1.1123904e-05]
 [ 1.1072329e-05  1.0143013e-05  1.0374155e-05 ...  8.4899748e-06
   9.4598918e-06  9.6567228e-06]]


In [54]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
#print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)

User latent shape:  (100, 100)
photo latent shape:  (88, 100)
Auxilary latent shape:  (88, 100)
Wu weight shape: (100, 100)
Wy weight shape: (88, 100)
Wa weight shape: (100, 100)
Wv weight shape: (100, 3374)
Beta shape: (100, 150)


# Testing Part

In [85]:
# without embedding
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        alpha[a]=(relu(np.dot(Au,np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa,
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av,np.expand_dims(all_3374[sample[a]],0).T)))*r
    mul=np.zeros((1,100))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], all_3374[k].T)
print(RS)

0 81


ValueError: setting an array element with a sequence.

In [118]:
#with Embedding
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.expand_dims(U[test_idx[s]],0).shape)
        #print(Au.shape)
        alpha[a]=(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T)))*r
        print('Relu---',np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))
    mul=np.zeros((1,100))
    
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], np.dot(E,all_3374[k].T))
#print(RS)

0 81
Relu--- [0.00016628]
Relu--- [-0.00011262]
Relu--- [0.00025554]
Relu--- [-0.00030319]
Relu--- [-0.00047679]
Relu--- [0.0002052]
alpha------------ [9.82560122e-05 0.00000000e+00 2.32311267e-05 0.00000000e+00
 0.00000000e+00 1.21252158e-04]
softmax alpha-------------- [0.1666763  0.16665992 0.1666638  0.16665992 0.16665992 0.16668013]
1 14
Relu--- [-0.00031382]
Relu--- [-2.28048077e-05]
Relu--- [0.00086642]
Relu--- [0.00029657]
alpha------------ [0.00000000e+00 0.00000000e+00 6.51441302e-06 2.96568532e-04]
softmax alpha-------------- [0.24998106 0.24998106 0.24998268 0.2500552 ]
2 3
Relu--- [-0.00765053]
Relu--- [0.00041229]
Relu--- [-0.00089143]
alpha------------ [0.         0.00041229 0.        ]
softmax alpha-------------- [0.33328752 0.33342496 0.33328752]
3 94
Relu--- [0.00013523]
Relu--- [-0.00011597]
Relu--- [-0.0001066]
Relu--- [-0.00028271]
Relu--- [0.00016099]
alpha------------ [0.00013523 0.         0.         0.         0.00016099]
softmax alpha-------------- [0.2000152 

In [93]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [94]:
target.shape

(10, 20)

In [95]:
testRS

array([[ 0.81236903, -0.58483609,  6.80391838,  5.16642127, -2.07160275,
        -1.07073596,  1.47381239, -6.71778664, -4.38768845,  0.62992083,
        -1.17663274, -2.8907774 , -5.39840717, -2.93548915,  2.69882834,
        -2.17588813,  0.40968546, -1.19366449,  2.86035374, -0.89758822],
       [ 4.71708011, -1.58780145, -1.03309382, -0.67662295,  3.10909413,
         5.28811126,  2.64583548,  0.48845324, -0.85946778, -2.68177119,
         3.79132009, 16.2498122 ,  4.95179154,  1.55345964,  3.20726294,
         5.59514535,  3.13114932,  1.72466598,  4.90734012,  1.51089048],
       [ 8.30910635,  7.73596105, 10.79921633,  5.14078609,  8.42743181,
        -1.1353207 , -2.82199001, -4.85774304,  0.03773887, -1.99882284,
         0.48202722, -7.56569824,  0.65614982,  0.15277922,  2.30333698,
         1.18460849, -1.90262803,  2.51180269, -3.06295594,  0.02852794],
       [ 4.01153543,  8.72515679, -1.05443674,  1.71226907,  4.38822361,
        -3.01381406, -1.41162891,  6.00718244, -

In [96]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

num of positive data in testing: 55.0
total testing data: 200


In [61]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [97]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    #print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
#print('avg_accuarcy for count_0:',avg_acc)
#print(acc_0)

In [63]:
acc_0

55

In [64]:
total

361

# Top 1 

In [98]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [99]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    print(top_0)
    count_0_all.append(top_0)
    #print(np.sum(target[i]))
    #print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

[2]
[11]
[2]
[1]
[14]
[0]
[7]
[2]
[17]
[4]


In [100]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.6 recall  0.10909090909090909


In [101]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

F1_score: 0.1846153846153846


# Top 3

In [102]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [103]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.5333333333333333 recall  0.2909090909090909


In [104]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

F1_score: 0.3764705882352941


# Top 5

In [105]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    print(top_5)
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

[2, 3, 18, 14, 6]
[11, 15, 5, 12, 18]
[2, 4, 0, 1, 3]
[1, 7, 4, 0, 3]
[14, 6, 15, 9, 2]
[0, 1, 2, 8, 13]
[7, 2, 18, 0, 10]
[2, 9, 5, 4, 12]
[17, 4, 18, 14, 9]
[4, 19, 12, 16, 15]


In [106]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

prec  0.46 recall  0.41818181818181815


In [107]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

F1_score: 0.43809523809523804


In [108]:
# pre_list
"""
test_amount = 150
yt_test_amount = 20
"""
all_sort = []
pre_matrix = np.zeros(shape=(test_amount,yt_test_amount))
for i in range(test_amount):
    top_5 = topN(list(testRS[i]),5) #取一個
    #print(top_5)
    all_sort.append(topN(list(testRS[i]),len(testRS[i])))
    for j in range(len(top_5)):
        pre_matrix[i][top_5[j]] = 1

In [109]:
pre_matrix.shape

(10, 20)

In [110]:
target.shape

(10, 20)

# NDCG

In [111]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
#Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
    idcg=0
    for i in range(len(ideal_list)):
        #print((2**true_list[i]-1),math.log2(i+2))
        idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
    #print('idcg',idcg)
    return idcg
def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg=0
    for i in range(len(prec_list)):
        dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
    #print('dcg',dcg)
    return dcg

In [112]:
total_ndcg = 0
num_ndcg = 5
for m in range(test_amount):
    idcg = IDCG([1]*num_ndcg)
    pre_list = []
    for s in all_sort[m][:num_ndcg]:
        #print(s)
        #print(target[m][s])
        pre_list.append(target[m][s])
    dcg = DCG(pre_list)
    ndcg = dcg/idcg
    #print(ndcg)
    total_ndcg += ndcg
avg_ndcg = total_ndcg/test_amount
print('NDCG:',avg_ndcg)

NDCG: 0.49306817004062264


# MAP

In [113]:
from sklearn.metrics import average_precision_score

In [114]:
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
total_prec = 0
for u in range(test_amount):
    y_true = target[u]
    y_scores = pre_matrix[u]
    total_prec+=average_precision_score(y_true, y_scores)
MAP = total_prec/test_amount

In [115]:
print('MAP',MAP)

MAP 0.46765151515151515
